In [90]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd
import numpy as np
import json
from ssl import create_default_context
from datetime import datetime
from datetime import timedelta
import requests
#from getpass import getpass
import parametros

## Parametros de conexion para ES

In [91]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
)

## tomando fecha mas reciente del indice

Las alertas que traiga el Endpoint con fechas menores a la maxima en indice, se descartan.

In [92]:
total_docs = 0
try:
    response = es.search(
        index= parametros.cambium_a_h_index,
        body={"aggs" : {
                   "max_date": {"max": {"field": "time_raised", "format": "yyyy-MM-dd HH:mm:ss"}}
                }
             },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["aggregations"]
    fecha_max=response["aggregations"]["max_date"]['value_as_string']
except:
    fecha_max = '2010-01-01 00:00:01'
print("Fecha maxima en indice:",fecha_max)

Fecha maxima en indice: 2021-10-05 23:17:54


In [93]:
now = datetime.now()
new_date = now - timedelta(days=6)
format = "%Y%m%d"
format_ES = "%Y.%m.%d"
fecha_url = str(new_date.strftime(format))
fecha_hoy = str(now.strftime(format_ES))

In [94]:
aaa = str(now.strftime("%Y%m%d%H"))+'00'
fecha_ini = (datetime.strptime(aaa, '%Y%m%d%H%M')-timedelta(minutes=60)).strftime("%Y%m%d%H%M")
fecha_fin = (datetime.strptime(fecha_ini, '%Y%m%d%H%M')+timedelta(minutes=59)).strftime("%Y%m%d%H%M")

## Leyendo la APi cambium-alarm-history

In [95]:
datos_api = pd.DataFrame(columns=['ip','network','message','name','severity','source_type','status'
                                          ,'time_raised','time_cleared','site','duration','id','code','mac'
                                          ,'acknowledged_by','source','managed_account'])




for k in range(0,len(parametros.url_cambium)):
    token_aux = 'Bearer ' + parametros.cambium_token_aux[k]
    cabecera1 = { 'Content-Type': 'application/json', 'Authorization' : token_aux, 'accept' : '*/*' } 
    #url = parametros.url_cambium[k] + "alarms/history" + '?start_time='+ fecha_url
    url = parametros.url_cambium[k] + "alarms/history" + '?start_time=' + fecha_ini + '&stop_time=' + fecha_fin
    print (url)
    print(token_aux)
    r = requests.get(url, headers = cabecera1, verify=False)
    if r.status_code == 200:
        res = json.loads(r.text)
        dato_param = res['paging']
        #se calcula los ciclos de la consulta paginada
        ciclos = int(round(dato_param['total']/100,0))
        i = 0
        while i < ciclos:
            i+=1
            offset = str(i)
            #url2 = parametros.url_cambium[k] + "alarms/history" + '?offset=' + offset  + '&start_time=' + fecha_url 
            url = parametros.url_cambium[k] + "alarms/history" + '?start_time=' + fecha_ini + '&stop_time=' + fecha_fin
            r = requests.get(url2, headers = cabecera1, verify=False)
            if r.status_code == 200:
                #print("respuesta 200: ",r.text)
                res = json.loads(r.text)
                datos_api = datos_api.append(res['data'], ignore_index=True)
            else:
                #break
                print("Error de request:",r.status_code)

        pass
    else:
        print("el else saca esto: ",r.status_code)
        #exit()

https://100.123.26.252/api/v2/alarms/history?start_time=202110052200&stop_time=202110052259
Bearer 78cc089920a918f3346e1198fab37ebf79b8cf72
el else saca esto:  401
https://100.123.26.224/api/v2/alarms/history?start_time=202110052200&stop_time=202110052259
Bearer 59c3b1fb3ed7b44b1cf84805929ba69483a1be66
el else saca esto:  401


/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.252'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.224'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [82]:
datos_api

,ip,network,message,name,severity,source_type,status,time_raised,time_cleared,site,duration,id,code,mac,acknowledged_by,source,managed_account


In [18]:
datos_api.to_excel("cambium_alarm_crudo.xlsx")

### Descartando datos de la API que ya están en el indice

In [7]:
try:
    datos_api['time_raised'] = (datos_api["time_raised"].str.split("T", n = 2, expand = True)[0])+' '+(datos_api["time_raised"].str.split("T", n = 2, expand = True)[1]).str.split("-", n = 1, expand = True)[0]
    datos_api['time_cleared'] = (datos_api["time_cleared"].str.split("T", n = 2, expand = True)[0])+' '+(datos_api["time_cleared"].str.split("T", n = 2, expand = True)[1]).str.split("-", n = 1, expand = True)[0]
except:
    print("Fecha:",now,"Error de request:",r.status_code)
    exit()

Error de request: 401


In [23]:
datos_api = datos_api.drop(datos_api[(datos_api['time_raised']<= fecha_max)].index)

In [49]:
datos_api.fillna('', inplace=True)

### Definiendo indice con fecha e insertando en ES

In [50]:
indice = parametros.cambium_a_h_index +'-'+ fecha_hoy

In [51]:
use_these_keys = ['ip','network','message','name','severity','source_type','status'
                  ,'time_raised','time_cleared','site','duration','id','code','mac'
                  ,'acknowledged_by','source','managed_account','@timestamp']
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

timestamp = datetime.now()
datos_api['@timestamp'] = timestamp.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(datos_api))
print("Fecha:",now,"- Total documentos insertados:",salida[0])

Fecha: 2021-05-10 17:40:44.237766 - Total documentos insertados: 0
